In [75]:
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

airbnb_files = ['data/raw_data/Austin_listings.csv', 'data/raw_data/Boston_listings.csv', 'data/raw_data/Asheville_listings.csv']


In [124]:
def drop_airbnb_cols(filename):
    df = pd.read_csv(filename)
    df.drop(['id', 'listing_url', 'scrape_id', 'last_scraped', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'calendar_last_scraped', 'weekly_price', 'monthly_price', 'neighbourhood_cleansed', \
             'license', 'jurisdiction_names', 'square_feet', 'neighbourhood', 'calculated_host_listings_count'], axis=1, inplace=True)
    return df

def get_col_names(files):
    counts = {}
    
    for f in files: 
        for c in drop_airbnb_cols(f).columns: 
            if c in counts: 
                counts[c] += 1
            else: 
                counts[c] = 1

    cols = []
    for c in counts: 
        if counts[c] == 3:
            cols.append(c)
    
    return cols

def segment(vector, train, dev, test): 
    count = len(vector)
    test += vector[-1*int(count*.1):]
    dev += vector[-1*int(count*.2):-1*int(count*.1)]
    train += vector[:int(count*.8)]
    
    return train, dev, test
    
def featurize(df):
    df['price'] = df['price'].map(lambda x: x.replace('$', "").replace(',',""))
    df[['price']] = df[['price']].apply(pd.to_numeric) # turn the price col into a number col
    prices = df['price'].tolist()
    text_cols = ['neighborhood_overview', 'space', 'description', 'host_about', 'notes', 'summary']
    X = [[prices[i]] for i in range(5)]
    for col in text_cols:
        corpus = df[col].fillna(value="").values
        vectorizer = CountVectorizer()
        x = vectorizer.fit_transform(corpus) #TODO: clean text
        for i, bow in enumerate(x[:5]):
            indices = bow.indices
            num = bow.data
            data = zip(indices, num)
            X[i] += [str(datum[0]) + ':' + str(datum[1]) for datum in data]
    return X

In [125]:
def create_datasets():
    col_names = get_col_names(airbnb_files)

    train = []
    dev = []
    test = []
    
    for f in airbnb_files:   
        df = pd.read_csv(f, dtype={'zipcode': 'str'})
        df = df[col_names]
        vector = featurize(df)
        train, dev, test = segment(vector, train, dev, test) # put data from f into train, dev, test
        
    return train, dev, test

In [126]:
def save_datasets(train, dev, test):
    with open('data/train_text.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(train)

#     with open('data/dev_text.csv', 'w') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows(dev)

#     with open('data/test_text.csv', 'w') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows(test)

In [127]:
a = _

In [128]:
save_datasets(*create_datasets())

X len bf:  5
X len after:  5
CHECK: 
X len bf:  5
X len after:  5
CHECK: 
X len bf:  5
X len after:  5
CHECK: 
12
